In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq groq==0.9.0 --progress-bar off

In [2]:
import json
import os
import sqlite3
from dataclasses import dataclass
from datetime import date
from enum import Enum, auto
from typing import List, Set

from google.colab import userdata
from groq import Groq

client = Groq(api_key=userdata.get("GROQ_API_KEY"))
MODEL = "llama3-groq-70b-8192-tool-use-preview"

## Habit Tracker App

In [3]:
DB_NAME = "habit_tracker.db"


class DayOfWeek(Enum):
    MONDAY = auto()
    TUESDAY = auto()
    WEDNESDAY = auto()
    THURSDAY = auto()
    FRIDAY = auto()
    SATURDAY = auto()
    SUNDAY = auto()


@dataclass
class Habit:
    id: int
    name: str
    repeat_frequency: Set[DayOfWeek]
    tags: List[str]


@dataclass
class DailyHabitEntry:
    id: int
    name: str
    tags: List[str]
    is_completed: bool


def get_connection():
    return sqlite3.connect(DB_NAME)


def create_tables():
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS habits (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                repeat_frequency TEXT NOT NULL,
                tags TEXT NOT NULL
            )
        """
        )
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS completions (
                habit_id INTEGER,
                completion_date TEXT,
                PRIMARY KEY (habit_id, completion_date),
                FOREIGN KEY (habit_id) REFERENCES habits (id)
            )
        """
        )
        conn.commit()


def list_habits() -> List[Habit]:
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM habits")
        return [
            Habit(
                id,
                name,
                {DayOfWeek[day] for day in freq.split(",")},
                tags.split(","),
            )
            for id, name, freq, tags in cursor.fetchall()
        ]


def habits_for_date(date: date) -> List[DailyHabitEntry]:
    weekday = DayOfWeek(date.weekday() + 1).name
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            SELECT h.id, h.name, h.tags, c.completion_date IS NOT NULL as completed
            FROM habits h
            LEFT JOIN completions c ON h.id = c.habit_id AND c.completion_date = ?
            WHERE instr(h.repeat_frequency, ?) > 0
        """,
            (date.isoformat(), weekday),
        )
        return [
            DailyHabitEntry(id, name, tags.split(","), bool(completed))
            for id, name, tags, completed in cursor.fetchall()
        ]


def complete_habit(habit_id: int, completion_date: date):
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT OR REPLACE INTO completions (habit_id, completion_date)
            VALUES (?, ?)
        """,
            (habit_id, completion_date.isoformat()),
        )
        conn.commit()


def add_habit(name: str, repeat_frequency: Set[DayOfWeek], tags: List[str] = []) -> int:
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT INTO habits (name, repeat_frequency, tags)
            VALUES (?, ?, ?)
        """,
            (name, ",".join(day.name for day in repeat_frequency), ",".join(tags)),
        )
        conn.commit()
        return cursor.lastrowid


def show_habits_for_date(date: date):
    print(f"Habits for {date}:")
    for entry in habits_for_date(date):
        status = "Completed" if entry.is_completed else "Not completed"
        print(f"- {entry.name} (ID: {entry.id}): {status}")
        print(f"  Tags: {', '.join(entry.tags)}")


create_tables()

In [4]:
add_habit(
    "Hit the gym",
    {DayOfWeek.MONDAY, DayOfWeek.WEDNESDAY, DayOfWeek.FRIDAY},
    ["exercise", "fitness"],
)
add_habit("Feed the llamas", {DayOfWeek.SATURDAY, DayOfWeek.SUNDAY}, ["diet"])

2

In [5]:
list_habits()

[Habit(id=1, name='Hit the gym', repeat_frequency={<DayOfWeek.FRIDAY: 5>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.MONDAY: 1>}, tags=['exercise', 'fitness']),
 Habit(id=2, name='Feed the llamas', repeat_frequency={<DayOfWeek.SATURDAY: 6>, <DayOfWeek.SUNDAY: 7>}, tags=['diet'])]

## Tools Definition

In [6]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "add_habit",
            "description": "Add a new habit. Returns the id for the habit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the habit",
                    },
                    "repeat_frequency": {
                        "type": "array",
                        "description": "Days of week to repeat, e.g. ['MONDAY', 'WEDNESDAY', 'FRIDAY']",
                        "items": {
                            "type": "string",
                            "enum": [
                                "MONDAY",
                                "TUESDAY",
                                "WEDNESDAY",
                                "THURSDAY",
                                "FRIDAY",
                                "SATURDAY",
                                "SUNDAY",
                            ],
                        },
                    },
                    "tags": {
                        "type": "array",
                        "description": "List of tags, e.g. ['health', 'fitness']",
                    },
                },
                "required": ["name", "repeat_frequency"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "list_habits",
            "description": "Returns a list of all available habits",
        },
    },
    {
        "type": "function",
        "function": {
            "name": "habits_for_date",
            "description": "Returns a list of habits scheduled for a date",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "str",
                        "description": "Date for which to display scheduled habits in ISO format e.g. 2024-11-23",
                    }
                },
                "required": ["date"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "complete_habit",
            "description": "Completes a specific habit for a given date",
            "parameters": {
                "type": "object",
                "properties": {
                    "habit_id": {
                        "type": "integer",
                        "description": "Id of the habit, e.g. 1",
                    },
                    "completion_date": {
                        "type": "str",
                        "description": "Date for completion of the habit in ISO format e.g. 2024-11-23",
                    },
                },
                "required": ["habit_id", "completion_date"],
            },
        },
    },
]

In [7]:
AVAILABLE_FUNCTIONS = {
    "add_habit": add_habit,
    "list_habits": list_habits,
    "habits_for_date": habits_for_date,
    "complete_habit": complete_habit,
}

## Map Arguments

In [8]:
argument_mapping = {}
argument_mapping["repeat_frequency"] = lambda day_names: [
    DayOfWeek[d] for d in day_names
]
argument_mapping["date"] = lambda d: date.fromisoformat(d)
argument_mapping["completion_date"] = lambda d: date.fromisoformat(d)

## Tool Prompting

In [9]:
user_prompt = "Add a new habit for Reading a book every weekday #learning"

messages = [
    {
        "role": "system",
        "content": "You are a personal assistant that helps with habit tracking. Use the provided functions to manage the execution of daily habits for the user.",
    },
    {
        "role": "user",
        "content": user_prompt,
    },
]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=TOOLS,
    tool_choice="auto",
    temperature=0,
    max_tokens=4096,
)

In [10]:
response_message = response.choices[0].message
messages.append(response_message)
print(response_message)
tool_calls = response_message.tool_calls
tool_calls

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hhwm', function=Function(arguments='{"name": "Reading a book", "repeat_frequency": ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY"], "tags": ["learning"]}', name='add_habit'), type='function')])


[ChatCompletionMessageToolCall(id='call_hhwm', function=Function(arguments='{"name": "Reading a book", "repeat_frequency": ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY"], "tags": ["learning"]}', name='add_habit'), type='function')]

In [11]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = AVAILABLE_FUNCTIONS[function_name]
    function_args = json.loads(tool_call.function.arguments)

In [12]:
function_args

{'name': 'Reading a book',
 'repeat_frequency': ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY'],
 'tags': ['learning']}

In [13]:
arguments_to_map = list(set(function_args.keys()) & set(argument_mapping.keys()))
arguments_to_map

['repeat_frequency']

In [14]:
for arg_name in arguments_to_map:
    function_args[arg_name] = argument_mapping[arg_name](function_args[arg_name])

In [15]:
print(function_args)

{'name': 'Reading a book', 'repeat_frequency': [<DayOfWeek.MONDAY: 1>, <DayOfWeek.TUESDAY: 2>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.THURSDAY: 4>, <DayOfWeek.FRIDAY: 5>], 'tags': ['learning']}


In [16]:
function_response = function_to_call(**function_args)
messages.append(
    {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": str(function_response),
    }
)
tool_response = client.chat.completions.create(
    model=MODEL, messages=messages, temperature=0
)

In [17]:
tool_message = tool_response.choices[0].message
messages.append(tool_message)
print(tool_message.content)

Your new habit "Reading a book" has been added successfully. It will repeat on weekdays and is tagged with #learning.


In [18]:
list_habits()

[Habit(id=1, name='Hit the gym', repeat_frequency={<DayOfWeek.FRIDAY: 5>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.MONDAY: 1>}, tags=['exercise', 'fitness']),
 Habit(id=2, name='Feed the llamas', repeat_frequency={<DayOfWeek.SATURDAY: 6>, <DayOfWeek.SUNDAY: 7>}, tags=['diet']),
 Habit(id=3, name='Reading a book', repeat_frequency={<DayOfWeek.THURSDAY: 4>, <DayOfWeek.FRIDAY: 5>, <DayOfWeek.MONDAY: 1>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.TUESDAY: 2>}, tags=['learning'])]

## Sequence of Function Calls

In [19]:
def map_arguments(function_args: dict, argument_mapping: dict = argument_mapping):
    arguments_to_map = list(set(function_args.keys()) & set(argument_mapping.keys()))
    for arg_name in arguments_to_map:
        function_args[arg_name] = argument_mapping[arg_name](function_args[arg_name])
    return function_args

In [20]:
def call_function(prompt, messages: List) -> List:
    messages.append(
        {
            "role": "user",
            "content": user_prompt,
        }
    )

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",
        temperature=0,
        max_tokens=4096,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    messages.append(response_message)

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = AVAILABLE_FUNCTIONS[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_args = map_arguments(function_args, argument_mapping)

        function_response = function_to_call(**function_args)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            }
        )
        tool_response = client.chat.completions.create(
            model=MODEL, messages=messages, temperature=0
        )
        tool_message = tool_response.choices[0].message
        messages.append(tool_message)
    return messages

In [21]:
today = date(2024, 7, 26)
today.isoformat(), DayOfWeek(today.weekday() + 1).name

('2024-07-26', 'FRIDAY')

In [22]:
show_habits_for_date(today)

Habits for 2024-07-26:
- Hit the gym (ID: 1): Not completed
  Tags: exercise, fitness
- Reading a book (ID: 3): Not completed
  Tags: learning


In [23]:
user_prompt = f"Show all habits for today - {today.isoformat()}"
print(user_prompt)

Show all habits for today - 2024-07-26


In [24]:
messages = call_function(user_prompt, messages)
print(messages[-1].content)

You have the following habits for today:
1. Hit the gym (Tags: exercise, fitness)
2. Reading a book (Tags: learning)


In [25]:
user_prompt = f"Complete the gym habit for {today.isoformat()}"
user_prompt

'Complete the gym habit for 2024-07-26'

In [26]:
messages = call_function(user_prompt, messages)
print(messages[-1].content)

The "Hit the gym" habit for today has been marked as completed.


In [27]:
show_habits_for_date(today)

Habits for 2024-07-26:
- Hit the gym (ID: 1): Completed
  Tags: exercise, fitness
- Reading a book (ID: 3): Not completed
  Tags: learning
